In [1]:
# Загружаем библиотеки
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import scikit_posthocs as sp
from math import factorial as fact

#conda install -c conda-forge scikit-posthocs
import scikit_posthocs as sp

#pip install cmh
from cmh import CMH

## <font color = green> Задание №1 </font>

Загрузите данные из файла 'file4_1.xlsx'. Постройте таблицу сопряженности (в строках укажите город).
Выведите маргинальную частоту для Москвы.

In [2]:
#Импортируем файл с данными
task_1 = pd.read_excel('file4_1.xlsx')

print('Количество наблюдений:', len(task_1))
task_1.head()

Количество наблюдений: 10000


,Fruit,City
0,Kiwi,Saint-Petersburg
1,Apple,Saint-Petersburg
2,Orange,Moscow
3,Pear,Saint-Petersburg
4,Pear,Kazan


In [3]:
#Создадим таблицу сопряженности
crosstab_1 = pd.crosstab(task_1['City'], task_1['Fruit'], rownames=['City'], colnames=['Fruit'])
crosstab_1

Fruit,Apple,Kiwi,Orange,Pear
City,,,,
Kazan,829,848,875,801
Moscow,868,809,855,783
Saint-Petersburg,858,805,806,863


In [4]:
#Выведем строку по Москве
crosstab_1[crosstab_1.index == 'Moscow']

Fruit,Apple,Kiwi,Orange,Pear
City,,,,
Moscow,868,809,855,783


In [5]:
#Рассчитаем маргинальную частоту по Москве
sum_1 = crosstab_1[crosstab_1.index == 'Moscow'].sum(axis=1)
sum_1

City
Moscow    3315
dtype: int64

## <font color = green> Задание №2 </font>

Есть ли зависимость между городом и предпочтением фрукта в данных из задания №1?
Подберите подходящий тест и проверьте эту гипотезу (приведите p-value и статистику).

In [12]:
#Выведем таблицу сопряженности
crosstab_1

Fruit,Apple,Kiwi,Orange,Pear
City,,,,
Kazan,829,848,875,801
Moscow,868,809,855,783
Saint-Petersburg,858,805,806,863


In [13]:
#Значений больше 50, а в ячейках нет экстремально маленьких значений (<5)
#То можно применять критерий Пирсона (хи-квадрат)

chi2_3, p_value_3, dof_3, expected_freq_3 = st.chi2_contingency(crosstab_1)

In [14]:
print('Статистика равна:', chi2_3)
print('P-value равен:', p_value_3)
print('Число степеней свободы:', dof_3)
print('Ожидаемые частоты:', np.round(expected_freq_3,0))

Статистика равна: 9.418181642422711
P-value равен: 0.15138955702360102
Число степеней свободы: 6
Ожидаемые частоты: [[857. 826. 850. 820.]
 [847. 816. 841. 811.]
 [851. 820. 845. 815.]]


In [15]:
#Найдем критическое значение для 95%
print('Критическое значение равно:', st.chi2.ppf(0.95, df=dof_3))

Критическое значение равно: 12.591587243743977


Не можем отвергнуть нулевую гипотезу об отсутствии зависимости между городом и предпочтением фрукта.

## <font color = green> Задание №3 </font>

С помощью точного критерия Фишера определите, зависит ли любовь к кофе от трудоустроенности человека.
P-value посчитайте с помощью формулы.

In [26]:
#Импортируем файл с данными
task_5 = pd.read_excel('file4_2.xlsx')

print('Количество наблюдений:', len(task_5))
task_5.head()

Количество наблюдений: 30


,Coffee,Work
0,Любит кофе,Работает
1,Любит кофе,Не работает
2,Любит кофе,Работает
3,Не любит кофе,Работает
4,Любит кофе,Не работает


In [27]:
#Создадим таблицу сопряженности
crosstab_5 = pd.crosstab(task_5['Coffee'], task_5['Work'], rownames=['Coffee'], colnames=['Work'])
crosstab_5

Work,Не работает,Работает
Coffee,,
Любит кофе,10,7
Не любит кофе,4,9


In [28]:
#Посчитаем маргинальные частоты:
s1 = crosstab_5.loc['Любит кофе'].sum()
s2 = crosstab_5.loc['Не любит кофе'].sum()
c1 = crosstab_5['Не работает'].sum()
c2 = crosstab_5['Работает'].sum()

s1, s2, c1, c2

(17, 13, 14, 16)

In [29]:
#Посчитаем значения в ячейках
a11 = crosstab_5['Не работает'][0]
a21 = crosstab_5['Не работает'][1]
a12 = crosstab_5['Работает'][0]
a22 = crosstab_5['Работает'][1]
n5 = a11+a21+a12+a22

a11, a21, a12, a22, n5

(10, 4, 7, 9, 30)

In [30]:
#Посчитаем p-value по формуле:
p_value_5 = (fact(s1)*fact(s2)*fact(c1)*fact(c2))/(fact(a11)*fact(a21)*fact(a12)*fact(a22)*fact(n5))
p_value_5

0.09562002624418785

## <font color = green> Задание №4 </font>

Проверьте гипотезу об отсутствии связи между анализируемыми качественными признаками с помощью критерия Кохрана-Мантеля-Хензеля.

In [38]:
task_6 = pd.DataFrame(
    [
        ['S1', 'no', 'yes'],
        ['S1', 'no', 'yes'],
        ['S1', 'no', 'yes'],
        ['S1', 'no', 'yes'],
        ['S1', 'yes', 'yes'],
        ['S1', 'yes', 'yes'],
        ['S1', 'yes', 'yes'],
        ['S1', 'yes', 'yes'],
        ['S2', 'yes', 'yes'],
        ['S2', 'yes', 'yes'],
        ['S2', 'yes', 'yes'],
        ['S2', 'no', 'yes'],
        ['S2', 'no', 'yes'],
        ['S2', 'no', 'yes'],
        ['S2', 'no', 'no'],
        ['S2', 'no', 'no'],

    ],
    columns=['stratum', 'A', 'B']
)

task_6

,stratum,A,B
0,S1,no,yes
1,S1,no,yes
2,S1,no,yes
3,S1,no,yes
4,S1,yes,yes
5,S1,yes,yes
6,S1,yes,yes
7,S1,yes,yes
8,S2,yes,yes
9,S2,yes,yes


In [39]:
result_6 = CMH(task_6, 'A', 'B', stratifier='stratum')
print(result_6)


        Cochran-Mantel-Haenszel Chi2 test

"A" x "B", stratified by "stratum"

Cochran-Mantel-Haenszel M^2 = 1.4, dof = 1, p-value = 0.2367



In [40]:
print('Число степеней свободы:', result_6.dof)
print('P-value:', result_6.p)

Число степеней свободы: 1
P-value: 0.2367235706378572
